In [30]:
import numpy as np
import itertools
import math

import operator

from collections import Counter, defaultdict

from scipy.spatial.distance import cityblock

In [2]:
test_input = """--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14"""

In [3]:
perms = [
[[1, 0, 0],
  [0, 1, 0],
  [0, 0, 1]],

 [[1, 0, 0],
  [0, 0, 1],
  [0, 1, 0]],

 [[0, 1, 0],
  [1, 0, 0],
  [0, 0, 1]],

 [[0, 1, 0],
  [0, 0, 1],
  [1, 0, 0]],
 
 [[0, 0, 1],
  [1, 0, 0],
  [0, 1, 0]],
 
 [[0, 0, 1],
  [0, 1, 0],
  [1, 0, 0]],
]

perms = [np.array(p) for p in perms]

flip = [-1, 1]
flip_1s = list(map(np.array, itertools.product(flip, flip, flip)))

rotation_matrices = [p*f for f in flip_1s for p in perms if np.linalg.det(p*f) == 1]

def column_set(a):
    return set(map(tuple, a.T))

def change_basis_f(rotation_number, diff):
    return lambda S: rotation_matrices[rotation_number] @ S + diff[:, None]

In [4]:
puzzle_input = open('inputs/19').read().strip()

def p1(puzzle_input):
    scanner_tups = []

    for scanner_s in puzzle_input.split('\n\n'):
        s = scanner_s.split('---')[2].strip().split('\n')
        tups = list(map(lambda t: np.array(tuple(map(int, t.split(',')))), s))
        scanner_tups.append(tups)
        
    scanner_matrices = [np.column_stack(tups) for tups in scanner_tups]
    
    def find_rotation_and_diff(s0, s1):
        s0_columns = column_set(s0)

        # try every orientation for s1
        for rotation_number, r in enumerate(rotation_matrices):
            changed = r @ s1

            # try to find difference vector which results in an overlap >= 12
            # we know that if there is an overlap that large there are 12 corresponding beacons
            # if we knew which ones corresponded we would have the difference vector.
            # without that, we can still try all pairs to find a correspondence
            for (b0, b1) in itertools.product(s0.T, changed.T):
                diff = b1 - b0
                tentative_transform = changed - diff[:, None]

                l = len(s0_columns & column_set(tentative_transform))

                if l >= 12:
                    return change_basis_f(rotation_number, -diff)
                
    adjacency_lists = defaultdict(list)
    
    for (i, j) in itertools.permutations(range(len(scanner_matrices)), 2):
        f = find_rotation_and_diff(scanner_matrices[i], scanner_matrices[j])

        if f:
            adjacency_lists[i].append((j, f))
            
    visited = set()
    beacons = set()
    scanners = []
    
    origin = np.array([[0], [0], [0]])
    
    def dfs(n, f):
        nonlocal beacons
        beacons |= column_set(f(scanner_matrices[n]))
                
        scanners.append(f(origin).flatten())
        visited.add(n)

        for (neighbor, g) in adjacency_lists[n]:
            if neighbor not in visited:
                dfs(neighbor, lambda S: f(g(S)))
                
    dfs(0, lambda x: x)
    
    return beacons, scanners

In [57]:
def p1(puzzle_input):
    scanner_tups = []

    for scanner_s in puzzle_input.split('\n\n'):
        s = scanner_s.split('---')[2].strip().split('\n')
        tups = list(map(lambda t: np.array(tuple(map(int, t.split(',')))), s))
        scanner_tups.append(tups)
        
    scanner_matrices = [np.column_stack(tups) for tups in scanner_tups]
    
    def find_rotation_and_diff(s0, s1):
        s0_columns = column_set(s0)

        # try every orientation for s1
        for rotation_number, r in enumerate(rotation_matrices):
            changed = r @ s1

            # try to find difference vector which results in an overlap >= 12
            # we know that if there is an overlap that large there are 12 corresponding beacons
            # if we knew which ones corresponded we would have the difference vector.
            # without that, we can still try all pairs to find a correspondence
            for (b0, b1) in itertools.product(s0.T, changed.T):
                diff = b1 - b0
                tentative_transform = changed - diff[:, None]

                l = len(s0_columns & column_set(tentative_transform))

                if l >= 12:
                    return change_basis_f(rotation_number, -diff)
            
    unvisited = set(range(len(scanner_matrices)))
    beacons = set()
    scanners = []
    
    origin = np.array([[0], [0], [0]])
    
    def dfs(n, f):
        print(unvisited)
        nonlocal beacons
        beacons |= column_set(f(scanner_matrices[n]))
                
        scanners.append(f(origin).flatten())
        unvisited.remove(n)
        
        for u in range(len(scanner_matrices)):
            if u not in unvisited:
                continue

            g = find_rotation_and_diff(scanner_matrices[n], scanner_matrices[u])
            
            print(f"Call from {n} to {u}")
            
            if g:
                dfs(u, lambda S: f(g(S)))
                
    dfs(0, lambda x: x)
    
    return beacons, scanners

In [62]:
beacons, scanners = p1(test_input)
assert len(beacons) == 79

{0, 1, 2, 3, 4}
Call from 0 to 1
{1, 2, 3, 4}
Call from 1 to 2
Call from 1 to 3
{2, 3, 4}
Call from 3 to 2
Call from 3 to 4
Call from 1 to 4
{2, 4}
Call from 4 to 2
{2}


In [60]:
math.comb(5, 2)

10

In [52]:
scanners

[array([0, 0, 0]),
 array([   68, -1246,   -43]),
 array([  -92, -2380,   -20]),
 array([  -20, -1133,  1061]),
 array([ 1105, -1205,  1229])]

In [53]:
max(cityblock(i, j) for (i, j) in itertools.combinations(scanners, 2))

3621

In [61]:
beacons, scanners = p1(puzzle_input)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26}
Call from 0 to 1
Call from 0 to 2
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26}
Call from 2 to 1
Call from 2 to 3
Call from 2 to 4
Call from 2 to 5
Call from 2 to 6
Call from 2 to 7
Call from 2 to 8
{1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26}
Call from 8 to 1
Call from 8 to 3
Call from 8 to 4
Call from 8 to 5
Call from 8 to 6
Call from 8 to 7
Call from 8 to 9
Call from 8 to 10
Call from 8 to 11
Call from 8 to 12
Call from 8 to 13
Call from 8 to 14
Call from 8 to 15
{1, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26}
Call from 15 to 1
Call from 15 to 3
Call from 15 to 4
Call from 15 to 5
Call from 15 to 6
Call from 15 to 7
Call from 15 to 9
Call from 15 to 10
Call from 15 to 11
Call from 15 to 12
{1, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 2

In [55]:
assert len(beacons) == 326

In [56]:
assert max(cityblock(i, j) for (i, j) in itertools.combinations(scanners, 2)) == 10630